# Лабораторна робота 3 з "Асиметричних криптосистем та протоколів"
## Тема: Криптосистема Рабіна. Атака на протокол доведення без розголошення.

**Виконали**\
Бондар Петро, ФІ-03\
Кістаєв Матвій, ФІ-03

In [25]:
import hashlib
import random as rnd
import requests
from optimus import *

In [26]:
KEY_LENGTH = 512


class Rabin_Server:
    __base_url = 'http://asymcryptwebservice.appspot.com/rabin/'
    s = requests.Session()
    n = None
    b = None
    
    # Setup server private key and receive server pub key
    def set_server_key(self, key_l: int) -> (str, str):
        req = f'{self.__base_url}serverKey?keySize={key_l}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        self.n, self.b = (int(r['modulus'], 16), int(r['b'], 16))
        return (self.n, self.b)
    
    # Ask server to encrypt
    def encrypt(self, M: str, rec_n, rec_b, type='TEXT'):
        req = f'{self.__base_url}encrypt?modulus={format(rec_n, "X")}&b={format(rec_b, "X")}&message={M}&type={type}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return (r['cipherText'], r['parity'], r['jacobiSymbol'])
    
    # Ask server to decrypt this message with his private keys
    def decrypt(self, C: str, p: int, j: int, type='TEXT'):
        req = f'{self.__base_url}decrypt?cipherText={C}&expectedType={type}&parity={p}&jacobiSymbol={j}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return r['message']
    
    # Ask server to sign this message with his private keys
    def sign(self, M: str, type='TEXT'):
        req = f'{self.__base_url}sign?message={M}&type={type}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return r['signature']
    
    # Verify the message using this public key
    def verify(self, M: str, sign: str, u_n, type='TEXT'):
        req = f'{self.__base_url}verify?message={M}&type={type}&signature={sign}&modulus={format(u_n, "X")}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return r['verified']


In [27]:
def str2hex(s: str):
    res = ""

    for c in s:
        cb = hex(ord(c))
        res += cb[2::]

    return res

def num2str(n: int):
    text = str()
    while n != 0:
        text += chr(n % 256)
        n //= 256
    
    return text[::-1]

def rabin_format(X: int):
    return 255*(2**(KEY_LENGTH*2 - 16)) + X * (2**64) + random.randrange(0, 2**64)

def rabin_unformat(X: int):
    return (X % (pow(2, (KEY_LENGTH*2 - 16)))) // (2**64)

In [28]:
class Rabin_User:
    def __init__(self, p, q, b = 0, serv = Rabin_Server()):
        print("Initializing user...")
        if not check_prime(p) or not check_prime(q):
            raise RuntimeError("p or q is not a prime number.")
        
        self.serv = serv
        self.p = p
        self.q = q
        self.b = b
        self.n = p*q
        self.get_server_public_key(KEY_LENGTH * 2)

        print(f"User private key (p, q): {(self.p, self.q)}")
        print(f"User public key (n, b): {(self.n, self.b)}")
        print(f"User server public key (n, b): {(self.serv.n, self.serv.b)}")
        print("--------------------------------------------------------")
    

    def get_server_public_key(self, len: int):
        self.serv.set_server_key(len)


    def send_message(self, M: str):
        print("Sending message to the server...")
        print(f"Sent message: {M}")

        M_format = rabin_format(int(str2hex(M), 16))
        
        c1 = ((M_format + (self.serv.b // 2)) % self.serv.n) % 2
        c2 = int(jacobi(M_format + (self.serv.b // 2), self.serv.n) == 1)
        C = format(M_format*(M_format + self.serv.b) % self.serv.n, "X")

        print(f"Sent cyphertext: {(C, c1, c2)}")
        

        M1 = self.serv.decrypt(C, c1, c2)
        print(f"Server responce: {M1}")

        check = (M1 == M)
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")


    def send_message_sign(self, M: str):
        print("Sending signature to the server...")
        
        X = rabin_format(int(str2hex(M), 16))
        while not (jacobi(X, self.p) == 1 and jacobi(X, self.q) == 1):
            X = ((X >> 64) << 64) + random.randrange(0, 2**64)

        Roots = sqrt_modpq(X, self.p, self.q)

        S = format(Roots[random.randrange(0, 4)], "X")

        check = self.serv.verify(M, S, self.n)

        print(f"Sent message: {M}")
        print(f"Sent signature: {S}")
        
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")


    def receive_message(self, M: str):
        print("Sending request for message to the server...")
        
        C, b1, b2 = self.serv.encrypt(M, self.n, self.b)
        Roots = sqrt_modpq(int(C, 16), self.p, self.q)

        for m in Roots:
            if [b1, b2] == [m % 2, (jacobi(m, self.p*self.q) == 1)]:
                M1 = m
                break
        
        M2 = (M1 % (pow(2, (KEY_LENGTH*2 - 16)))) // (2**64)
        M2 = num2str(M2)

        check = (M == M2)

        print(f"Sent message: {M}")
        print(f"Received cyphertext: {C}")
        print(f"Decoded cyphertext: {M2}")
        
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")
        
        
    def receive_message_sign(self, M: str):
        print("Sending request for message signature to the server...")
        
        S = self.serv.sign(M)
        M1 = num2str(rabin_unformat(pow(int(S, 16), 2, self.serv.n))) 
        
        check = (M1 == M)

        print(f"Sent message: {M}")
        print(f"Received signature: {S}")
        print(f"Message signed with signature: {M1}")
        
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")


In [29]:
print(f"Довжина модуля сервера: {2*KEY_LENGTH}")
print(f"Ключі для користувача 1 ({KEY_LENGTH} бітів для кожного простого):")
p1 = generate_blum_prime(KEY_LENGTH)
print(f"p1 = {p1}")
q1 = generate_blum_prime(KEY_LENGTH, [p1])
print(f"q1 = {q1}")

Довжина модуля сервера: 1024
Ключі для користувача 1 (512 бітів для кожного простого):
p1 = 8724225277736938084736640372971249846977911532213977294393134315844068341001829247525197651279012365659583613487006605597360003890368301329626066319166203
q1 = 6910321698978849146057836685075489773623200606030578617175894005099932117634979358532793079454991139442975631040837074159151292288908904744874877746958791


In [30]:
u_r = Rabin_User(p1, q1)
u_r.send_message("sdjdfgjdhgfjdgfhj")
# u_r.receive_message("Hello!")
# u_r.send_message_sign("Hello!")
# u_r.receive_message_sign("Hello!")

Initializing user...
H: Sent request: http://asymcryptwebservice.appspot.com/rabin/serverKey?keySize=1024
S: Response: {'b': '6E6498CF530FF0EF289C70B6AD80F89E7A8D2FD022999522D77D468E509CBCF3931816F06B63399FE7177222C38824FEE0C0475927502A718824AA9AE7243016CA5530BE491AFD6AD66260C0962997C517B2D9E446818AD302B6C297C22A3CCB379019B7D628A60BD283527CD9066CF1DECFD7E1FC30FDFD4B06EF5A70689A73', 'modulus': 'B2F65E7AA6B99FDD31E44D0BB590B580CEDC968FCBBA739C77E13CB4E64F0812A161E27AC58DA77218B34C3E05088A637520BEF3BA543E6775A492BA7D1F87A467C0072B3623C14DB81C726C04CA25D041CD88BECAE5CE136D9FA5E4BCA3ADB81EE43F0EBE45A9A7E4C6E4B249C2B795EB8DC1C102C2216AB806B29F8D059681'}
User private key (p, q): (8724225277736938084736640372971249846977911532213977294393134315844068341001829247525197651279012365659583613487006605597360003890368301329626066319166203, 6910321698978849146057836685075489773623200606030578617175894005099932117634979358532793079454991139442975631040837074159151292288908904744874877746958791)
User p

RuntimeError: Incorrect server status code 400:
	Request http://asymcryptwebservice.appspot.com/rabin/decrypt?cipherText=295255C12E851F0281D9CF65A4F1FCCBAA6BFC3111218A4A64EBD9C6D669BD565D26A209DD5F1574F229BFB546C041A89290BA25F125E25F1042C83EBFB9A11BC99145E1DD8B1AD898A8E35C842D27301B4253A3AABD966BCDA1F7DF98AF87171502E64DE4DD7BE07F0563663169419BE74B426A7FD7960542AF855CB2D3862E&expectedType=TEXT&parity=1&jacobiSymbol=1
	Response{'message': 'Invalid padding'}

In [ ]:
class ZNP_Server:
    __base_url = 'http://asymcryptwebservice.appspot.com/znp/'
    s = requests.Session()
    n = None
    
    # Setup server private key and receive server pub key
    def set_server_key(self) -> (str, str):
        req = f'{self.__base_url}serverKey'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        self.n = int(r['modulus'], 16)
        return self.n
    
    def take_sqrt_mod_n(self, y):
        req = f'{self.__base_url}challenge?y={format(y, "X")}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return r['root']


class ZNP_User:
    def __init__(self, serv = ZNP_Server()):
        print("Initializing user...")
        
        self.serv = serv
        self.get_server_public_key()

        print(f"User server public key (n): {self.serv.n}")
        print("--------------------------------------------------------")

    def get_server_public_key(self):
        self.serv.set_server_key()

    # Ask server to 
    def attack_server(self):
        print("Starting ZNP attack...")

        print(f"Server public key: {self.serv.n}")

        itr = 0
        while True:
            x1 = rnd.randint(1, self.serv.n)
            itr += 1

            if jacobi(x1, self.serv.n) != -1:
                continue
            print(f"Randomed {itr} times")

            print("\n==== Asking to take root =====")
            x2 = int(self.serv.take_sqrt_mod_n(pow(x1, 2, self.serv.n)), 16)
            print("========================")
            print(f"\nCandidate: {x2}")
            
            if x1 != x2 and x1 != ((-x2) % self.serv.n):
                print("Candidate is OK!")
                break

        p = math.gcd(x1 - x2, self.serv.n)
        if p == 1:
            p = math.gcd(x1 + x2, self.serv.n)
        q = self.serv.n // p

        print("\n== Results ==")
        print(f"p: {p}")
        print(f"q: {q}")
        print(f"p*q: {p*q}")
        print("===========\n")

        check = (p*q == self.serv.n)
        if check:
            print("Success!")
        else:
            print("No Success. ЩЗХ? :(")
        
        print("--------------------------------------------------------")

In [ ]:
znp_u = ZNP_User()
znp_u.attack_server()

Initializing user...
H: Sent request: http://asymcryptwebservice.appspot.com/znp/serverKey
S: Response: {'modulus': 'A1E3697C6CC311838B01C9BDBECD1D09F15F3F6EE74123A3B6F1DDF35ACF6E80C6CD612894179BB50351FCAB65F0ED943C183106529B037E1106AE8F5F5B144FF77413D534AF8827EBFDDD5DEC44793D12341A69B7EA429E83EF44D20B541173DD06C06E21634031E9425EC75AEE1518EAD5BEDF6294E26E054BF63AD2B3A561686418FC38A34369242D0137E1852B2E5B221C1034F577B0A8C0D501833BBE8D6E8F0EC2765C7255E93077D31FF8115D2E8D9F633D1C99E17266C9F2242E34DB7A303FB51E87665E7688497E5FF26AC5BF8C476E05D6B3330359D820B14B0356DFA2AC66E7E0F3B13EA59AF7F7BA1754A4E5EEDE962EBC38F5ABD91268357599'}
User server public key (n): 2043650816332312875955614301234275062507875754711464608324827975196694945811519470932700774705197288780058415054745859053718197483388521884509362990174559068526621455695886333560985349475968681702785400672007761634401418151356801742506317150368276548258287829886181147839808790603235982756854997786351387202344887233270438205586817790911417